In [39]:
# Import dependencies
import pandas as pd
import warnings
warnings.filterwarnings("ignore", message="Could not infer format, so each element will be parsed individually")

In [40]:
# Set the path to the file containing data stes
path_feb = 'Resources/JC-202402-citibike-tripdata.csv'
path_mar = 'Resources/JC-202403-citibike-tripdata.csv'
path_apr = 'Resources/JC-202404-citibike-tripdata.csv'

In [41]:
# Read files
trip_data_feb = pd.read_csv(path_feb)
trip_data_mar = pd.read_csv(path_mar)
trip_data_apr = pd.read_csv(path_apr)

In [42]:
# Verify if columns name are identical across all data sets

# Get the column headers from each dataset
columns_feb = set(trip_data_feb.columns)
columns_mar = set(trip_data_mar.columns)
columns_apr = set(trip_data_apr.columns)

# Compare the columns across the datasets
common_columns = columns_feb.intersection(columns_mar, columns_apr)
feb_unique_columns = columns_feb - common_columns
mar_unique_columns = columns_mar - common_columns
apr_unique_columns = columns_apr - common_columns

# print columns in our data sets that do not match
print("\nColumns in February Data that do not match other months:")
print(feb_unique_columns)

print("\nColumns in March Data that do not match other months:")
print(mar_unique_columns)

print("\nColumns in April Data that do not match other months:")
print(apr_unique_columns)
print("If the sets are empty, it means all column names are identical.")


Columns in February Data that do not match other months:
set()

Columns in March Data that do not match other months:
set()

Columns in April Data that do not match other months:
set()
If the sets are empty, it means all column names are identical.


In [43]:
# Combine the DataFrames
combined_data = pd.concat([trip_data_feb, trip_data_mar, trip_data_apr])
# Reset the index for the combined data
combined_data.reset_index(drop=True, inplace=True)

In [44]:
# Show columns and their data types as a first step of cleaning the data
print(combined_data.dtypes)

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object


In [45]:
# Convert columns to datetime format from currecnt 'object' type
# The exact time is left as this information may be useful later
combined_data['started_at'] = pd.to_datetime(combined_data['started_at'])
combined_data['ended_at'] = pd.to_datetime(combined_data['ended_at'])




In [46]:
# Create a new column showing the duration of each ride
combined_data['duration_minutes'] = ((combined_data['ended_at'] - combined_data['started_at']).dt.total_seconds() / 60).astype(int)

# Verify if datetime was changed correctly and duration is right
print(combined_data.dtypes[['started_at','ended_at','duration_minutes']])
print(combined_data.head(5))

started_at          datetime64[ns]
ended_at            datetime64[ns]
duration_minutes             int32
dtype: object
            ride_id  rideable_type          started_at            ended_at  \
0  A841EF9C3617E47B  electric_bike 2024-02-12 16:17:19 2024-02-12 16:23:24   
1  514881A61CAD0A93  electric_bike 2024-02-04 15:04:16 2024-02-04 15:40:14   
2  511F37086CCC4510  electric_bike 2024-02-16 16:23:16 2024-02-16 16:27:16   
3  27A2AA8BDD0D1F6E   classic_bike 2024-02-25 17:05:01 2024-02-25 17:10:53   
4  2808ABEC0903C18E  electric_bike 2024-02-29 15:21:15 2024-02-29 15:26:03   

       start_station_name start_station_id            end_station_name  \
0         Adams St & 2 St            HB407  Stevens - River Ter & 6 St   
1         Adams St & 2 St            HB407  Stevens - River Ter & 6 St   
2  Clinton St & Newark St            HB409  Stevens - River Ter & 6 St   
3         Adams St & 2 St            HB407  Stevens - River Ter & 6 St   
4         Adams St & 2 St            HB407

In [47]:
# Display the number of rows in combined_data
number_rows = len(combined_data)
print(f"Number of rows in combined_data: {number_rows}")

Number of rows in combined_data: 200310


In [48]:
# Check for duplicate rows
duplicates = combined_data.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [49]:
# Check potential misspells in the station names:

# Group the data by 'station_id' and 'station_name'
# Then check if each 'station_id' is associated with more than one 'station_name'

# Group by 'start_station_id' and count the number of unique 'start_station_name' for each ID
start_station_consistency = combined_data.groupby('start_station_id')['start_station_name'].nunique()
end_station_consistency = combined_data.groupby('end_station_id')['end_station_name'].nunique()

# Filter for station IDs that have more than one associated station name
inconsistent_start_stations = start_station_consistency[start_station_consistency > 1]
inconsistent_end_stations = end_station_consistency[end_station_consistency > 1]

# To see the specific rows with inconsistencies for start stations
if not inconsistent_start_stations.empty:
    inconsistent_rows = combined_data[combined_data['start_station_id'].isin(inconsistent_start_stations.index)]
    print("\nRows with inconsistent start station names for the same station ID:")
    print(inconsistent_rows)
else:
    print("All start station IDs are consistently associated with a single start station name.")
# To see the specific rows with inconsistencies for start stations
if not inconsistent_end_stations.empty:
    inconsistent_end_rows = combined_data[combined_data['start_station_id'].isin(inconsistent_end_stations.index)]
    print("\nRows with inconsistent start station names for the same station ID:")
    print(inconsistent_end_rows)
else:
    print("All end station IDs are consistently associated with a single start station name.")

All start station IDs are consistently associated with a single start station name.
All end station IDs are consistently associated with a single start station name.


In [50]:
# Check if station IDs match with station names 
inconsistent_start_stations = combined_data[combined_data['start_station_name'].isna() & combined_data['start_station_id'].notna()]
inconsistent_end_stations = combined_data[combined_data['end_station_name'].isna() & combined_data['end_station_id'].notna()]

print(f"Inconsistent start stations: {len(inconsistent_start_stations)}")
print(f"Inconsistent end stations: {len(inconsistent_end_stations)}")

Inconsistent start stations: 0
Inconsistent end stations: 0


In [51]:
# Check for invalid latitude and longitude values
invalid_start_coords = combined_data[(combined_data['start_lat'] < -90) | (combined_data['start_lat'] > 90) |
                                     (combined_data['start_lng'] < -180) | (combined_data['start_lng'] > 180)]
invalid_end_coords = combined_data[(combined_data['end_lat'] < -90) | (combined_data['end_lat'] > 90) |
                                   (combined_data['end_lng'] < -180) | (combined_data['end_lng'] > 180)]

print(f"Invalid start coordinates: {len(invalid_start_coords)}")
print(f"Invalid end coordinates: {len(invalid_end_coords)}")

Invalid start coordinates: 0
Invalid end coordinates: 0


In [52]:
# Check if any rides end before they start (incorrect date records)
invalid_dates = combined_data[combined_data['ended_at'] < combined_data['started_at']]
print(f"Rides that end before they start: {len(invalid_dates)}")

Rides that end before they start: 0


In [53]:
# Check if any other mispelling in the data in 'casual_member' column
invalid_member_casual = combined_data[~combined_data['member_casual'].isin(['casual', 'member'])]
# Display the rows with invalid entries
print(invalid_member_casual)

Empty DataFrame
Columns: [ride_id, rideable_type, started_at, ended_at, start_station_name, start_station_id, end_station_name, end_station_id, start_lat, start_lng, end_lat, end_lng, member_casual, duration_minutes]
Index: []


In [54]:
# Check if the data is missing any values
missing_values = combined_data.isnull().sum()
# Show the columns with missing values
print(missing_values[missing_values > 0])

start_station_name     99
start_station_id       99
end_station_name      554
end_station_id        583
end_lat                54
end_lng                54
dtype: int64


In [55]:
# Drop rows with any missing values
combined_data_cleaned = combined_data.dropna()
# Ensure the data no longer has any missing values
missing_data = combined_data_cleaned.isnull().sum()
# Show the columns with missing values
print(missing_data[missing_data > 0])

Series([], dtype: int64)


In [56]:
# Display the number of rows in combined_data_cleaned
number_rows_clean = len(combined_data_cleaned)
print(f"Number of rows in combined_data: {number_rows_clean}")

Number of rows in combined_data: 199674


In [57]:
# Display first 5 rows in our data
print(combined_data_cleaned.head(5))
# Save cleaned data to csv file
combined_data_cleaned.to_csv('Output_Files/tripdata_feb_to_apr_2024.csv', index = False)

            ride_id  rideable_type          started_at            ended_at  \
0  A841EF9C3617E47B  electric_bike 2024-02-12 16:17:19 2024-02-12 16:23:24   
1  514881A61CAD0A93  electric_bike 2024-02-04 15:04:16 2024-02-04 15:40:14   
2  511F37086CCC4510  electric_bike 2024-02-16 16:23:16 2024-02-16 16:27:16   
3  27A2AA8BDD0D1F6E   classic_bike 2024-02-25 17:05:01 2024-02-25 17:10:53   
4  2808ABEC0903C18E  electric_bike 2024-02-29 15:21:15 2024-02-29 15:26:03   

       start_station_name start_station_id            end_station_name  \
0         Adams St & 2 St            HB407  Stevens - River Ter & 6 St   
1         Adams St & 2 St            HB407  Stevens - River Ter & 6 St   
2  Clinton St & Newark St            HB409  Stevens - River Ter & 6 St   
3         Adams St & 2 St            HB407  Stevens - River Ter & 6 St   
4         Adams St & 2 St            HB407  Stevens - River Ter & 6 St   

  end_station_id  start_lat  start_lng    end_lat    end_lng member_casual  \
0       